<a href="https://colab.research.google.com/github/Sonniah509/Demo-repo21/blob/master/variationalautoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import torch
from torch import nn
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Lambda
import keras.backend as K
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
def preprocess_images(images):
  images = images.reshape((images.shape[0], 28, 28, 1)) / 255.
  return np.where(images > .5, 1.0, 0.0).astype('float32')

x_train = preprocess_images(x_train)
x_test = preprocess_images(x_test)
train_size = 6000
batch_size = 32
test_size = 1000
x_train = (tf.data.Dataset.from_tensor_slices(x_train).shuffle(train_size).batch(batch_size))
x_test = (tf.data.Dataset.from_tensor_slices(x_test).shuffle(test_size).batch(batch_size))
latent_dim = 2
#encoder
inputs = keras.Input(shape=(28, 28, 1))
x = Conv2D(32, 3, padding='same', activation='relu')(inputs)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Flatten()(x)
mu = Dense(latent_dim, activation="relu")(x)
log_var = Dense(latent_dim, activation = "relu")(x)
# Reparameterization trick
epsilon = K.random_normal(shape=(latent_dim,))
z = mu + tf.exp(0.5 * log_var) * epsilon

decoder_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(decoder_input)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)

encoder = keras.Model(inputs, z)
decoder = keras.Model(decoder_input, decoder_output)

# VAE model
vae_output = decoder(z)
vae = keras.Model(inputs, vae_output)
# Define the VAE loss function
kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(mu) - tf.exp(log_var), axis=-1)
reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(vae_output)) * 28 * 28)
vae_loss = reconstruction_loss + 0.1 * kl_loss
# Compile the VAE model
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.fit(x_train, epochs=10, batch_size=128)
gen_im = vae.predict(x_test)
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(10, 1))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(gen_im[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
